## Conformalized Survival Analysis with Right-Censoring

This notebook demonstrates the use of DR-COSARC on real data sets.

In [24]:
# Load required libraries
suppressMessages(library(tidyverse))
suppressMessages(library(gridExtra))
library(survival)
library(ggsurvfit)
library(grf)
library(cfsurvival)
suppressMessages(library(hdf5r))

## Load the data

In [25]:
source("utils_real_data.R")

for(data.name in c("VALCT", "PBC", "GBSG", "METABRIC")) {   
    cat(sprintf("Processing %s...\n", data.name))
    data <- load_data(data.name)
    data %>% write_csv(sprintf("../../data/data_csv/%s.csv", data.name))
}

Processing VALCT...
Processing PBC...
Processing GBSG...
Processing METABRIC...


Rows: 2509 Columns: 39
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (27): Study ID, Patient ID, Sample ID, Type of Breast Surgery, Cancer Ty...
dbl (12): Age at Diagnosis, Cohort, Neoplasm Histologic Grade, Lymph nodes e...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  4 ; coefficient may be infinite. ”
